In [2]:
import pandas as pd
import json
import os

## Fabricated Benchmark

In [3]:
def read_json_file(path, new_dir):
    json_files = [pos_json for pos_json in os.listdir(path) if "mapping" in pos_json]
    if not json_files:
        return
    json_file = json_files[0]
    
    tables = [table for table in os.listdir(path) if table.endswith(".csv")]
    
    source_tab = [table for table in tables if "source" in table][0]
    target_tab = [table for table in tables if "target" in table][0]
    
    df_dict = {
        "source_col": [],
        "target_col": [],
        "source_tab": [],
        "target_tab": []
    }
    
    with open(os.path.join(path, json_file)) as f:
        d = json.load(f)

        for match in d['matches']:
            print(match["source_column"], match["target_column"])
            df_dict["source_col"].append(match["source_column"])
            df_dict["target_col"].append(match["target_column"])
            df_dict["source_tab"].append(source_tab.split(".")[0])
            df_dict["target_tab"].append(target_tab)
            

    source_df = pd.read_csv(os.path.join(path, source_tab))
    target_df = pd.read_csv(os.path.join(path, target_tab))
    source_df.to_csv(os.path.join(new_dir, "source-tables", source_tab), index=False)
    target_df.to_csv(os.path.join(new_dir, "target-tables", target_tab), index=False)

    return pd.DataFrame(df_dict)

In [38]:
types = ["Joinable", "Semantically-Joinable", "Unionable", "View-Unionable"]
datasets = ["ChEMBL", "OpenData", "TPC-DI", "Wikidata"]

for dataset in datasets:
    print(dataset)
    for type in types:
        print(type)
        root = "./Valentine-datasets/" + dataset + "/" + type
        new_dir = "./valentine/" + dataset + "/" + type

        if not os.path.exists(new_dir):
            os.makedirs(new_dir)
            os.makedirs(new_dir + "/source-tables")
            os.makedirs(new_dir + "/target-tables")

        ground_truth = pd.DataFrame(columns=["source_col", "target_col", "source_tab", "target_tab"])

        for _, dirs, _ in os.walk(root):
            for dir in dirs:
                print(dir)
                df = read_json_file(os.path.join(root, dir), new_dir)
                if df is None:
                    continue

                ground_truth = pd.concat([ground_truth, df], ignore_index=True)
                
        if dataset == "Wikidata":
            df = read_json_file(root, new_dir)
            ground_truth = pd.concat([ground_truth, df], ignore_index=True)

        ground_truth.to_csv(os.path.join(new_dir, "matches.csv"), index=False)
            
        

Wikidata
Joinable
musician musicianID
birthDate birthDate
familyNameLabel familyName
givenNameLabel forename
numberOfChildren NChildren
websiteLabel webpage
Semantically-Joinable
musician musicianID
musicianLabel musicianName
genderLabel genderType
cityLabel city
fatherLabel fatherName
motherLabel motherName
partner spouse
genreLabel kind
Unionable
musician musicianID
musicianLabel musicianName
genderLabel genderType
birthDate birthDate
cityLabel city
familyNameLabel familyName
givenName forename
fatherLabel fatherName
motherLabel motherName
partner spouse
numberOfChildren NChildren
genreLabel kind
websiteLabel webpage
residenceLabel residence
ethnicityLabel ethnicity
religionLabel religionLabel
activityStart kickoff
twitterNameLabel twitterUsername
geniusNameLabel geniusNameLabel
recordLabelLabel recordCompany
View-Unionable
musician musicianID
birthDate birthDate
familyNameLabel familyName
givenName forename
numberOfChildren NChildren
websiteLabel webpage


In [4]:
dataset = "Magellan"
root = "./Valentine-datasets/" + dataset
new_dir = "./valentine/" + dataset

if not os.path.exists(new_dir):
    os.makedirs(new_dir)
    os.makedirs(new_dir + "/source-tables")
    os.makedirs(new_dir + "/target-tables")

ground_truth = pd.DataFrame(columns=["source_col", "target_col", "source_tab", "target_tab"])

for _, dirs, _ in os.walk(root):
    for dir in dirs:
        print(dir)
        df = read_json_file(os.path.join(root, dir), new_dir)
        if df is None:
            continue

        ground_truth = pd.concat([ground_truth, df], ignore_index=True)

ground_truth.to_csv(os.path.join(new_dir, "matches.csv"), index=False)
    
        

amazon_google_exp
id id
title title
manufacturer manufacturer
price price
dblp_acm
id id
title title
authors authors
venue venue
year year
walmart_amazon
id id
title title
category category
brand brand
modelno modelno
price price
dblp_scholar
id id
title title
authors authors
venue venue
year year
beeradvo_ratebeer
id id
Beer_Name Beer_Name
Brew_Factory_Name Brew_Factory_Name
Style Style
ABV ABV
itunes_amazon
id id
Song_Name Song_Name
Artist_Name Artist_Name
Album_Name Album_Name
Genre Genre
Price Price
CopyRight CopyRight
Time Time
Released Released
fodors_zagats
id id
name name
addr addr
city city
phone phone
type type
class class


## Biomedical Pretrain: Create joined dataset

In [1]:
import os
import pandas as pd

usecases = ["cao", "dou", "clark", "huang", "krug", "satpathy", "vasaikar", "wang"]

df_all = pd.DataFrame({})
for usecase in usecases:
    df = pd.read_csv(os.path.join(usecase, "source.csv")).sample(50)
    df_all = pd.concat([df_all, df], sort=False)
df_all

df_all.to_csv("gdc_all/source.csv", index=False)



In [2]:
train_usecases = ["cao", "dou", "clark", "huang"]

gt_all = pd.DataFrame({})

for usecase in train_usecases:
    df = pd.read_csv(os.path.join(usecase, "groundtruth.csv"))
    gt_all = pd.concat([gt_all, df], sort=False)
gt_all = gt_all.drop_duplicates()
gt_all

gt_all.to_csv("gdc_all/groundtruth.csv", index=False)